<a href="https://colab.research.google.com/github/pankosmas/TopicExtractor/blob/main/Greek-BERT%20Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
!pip install bertopic
!pip install scikit-learn
!pip install --upgrade joblib==1.1.0
!pip install tensorflow
!pip install seaborn

from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, silhouette_score
from sklearn.cluster import KMeans

In [ ]:
from bertopic import BERTopic
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import string
import re
import hdbscan
import umap
from umap import UMAP 
from hdbscan import HDBSCAN
from mpl_toolkits.mplot3d import Axes3D


import pandas as pd
import numpy as np

import tensorflow as tf
import transformers #huggingface transformers library
 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import sklearn
from sklearn.metrics import confusion_matrix

import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
sns.set(style='white', context='poster', rc={'figure.figsize':(14,10)})

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None
if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()
print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


# OMILIES TOPIC MODEL

In [ ]:
stoplist = set('αλλα αλλά εγω εγώ εσυ εσύ αυτος αυτός αυτη αυτή αυτο αυτό εμεις εμείς εσεις εσείς αυτοι αυτοί αυτα '
               'αυτά το του τα των τις τους τοις και ναι οχι μη μην δε λίγο λιγο τόσο τοσο γι για δεν ειμαι είμαι '
               'εισαι είσαι ειναι είναι ειμαστε είμαστε ειστε είστε ειναι είναι ουτε ούτε μητε μήτε ουδε ουδέ η ή '
               'ειτε είτε αν και μα παρα πάρα παρά ομως όμως ωστοσο ωστόσο ενω ενώ μολονοτι μολονότι μονο μόνο μονό '
               'που λοιπον λοιπόν ωστε ώστε αρα άρα επομενως επομένως οποτε όποτε οπότε δηλαδη δηλαδή οτι ότι πως που '
               'μην μηπως μήπως να αμα άμα οταν όταν καθως καθώς αφου αφού αφοτου αφότου πριν μολις προτου ωσπου '
               'ωσοτου σαν γιατι διοτι επειδη αφου τι οτι για να ωστε ως παρα αναμεσα κάνω κάνεις κάνει κάνουν '
               'κάνουμε κι δε μέρα μεταξυ εαν ανω κατω πανω πισω μπρος μπροστα εχω κανω λεω βλεπω μπορω μπορει ισως '
               'καλα καλο καλος καλη καλων έχω έχεις έχει έχουμε έχετε έχουν εχω εχεις εχει εχουμε εχετε εχουν'
               'είχα είχες είχε είχαμε είχατε είχαν ειχα ειχες ειχε ειχαμε ειχατε ειχαν καλους καλε εκει εκτος εντος'
               'μεσα εξω ιδιο ηταν ζωη ολα ολο ολος ολοι ομως ποτε σπανια οποιος οποια οποιο οποιους οποιοι οποιες πολυ πολλα πολλη πολλων πολλους τωρα χθες σημερα '
               'αυριο παρον παρων μελλον παρελθον χθες ωρα χωρις με χρονια πρωτος της κατα στα στο στη στων στις στους '
               'οι ο η απο στην στη στον στο τον την μόλις προτού ώσπου ωσότου σαν γιατί διότι επειδή αφού τι ότι για '
               'να ώστε ως πάρα αναμεσά έχετε κάνετε χθες χτες εχτές εχθές μεταξύ εάν άνω κάτω πάνω πίσω μπρος μπροστά '
               'έχω κάνω λέω βλέπω μπορώ μπορεί ίσως καλά καλό καλός καλή καλών καλούς καλέ εκεί εκτός εντός μέσα έξω '
               'ίδιο ήταν ζωή μεγάλη μικρή όλα όλο όλος όλοι όμως ποτέ σπάνια όποιος όποια όποιο όποιους όποιοι όποιες '
               'πολύ πολλά πολλή πολλών πολλούς τώρα χθες σήμερα αύριο παρόν παρών μέλλον παρελθόν χθες ώρα χωρίς με '
               'χρονιά πρώτος της κατά στα στο μετα μετά όσων οσων στη στων στις στους οι ο η από στην στη στον στο τον την θα όμως σε '
               'αυτού τη όλους μας σας πρέπει ήδη έχει είχε μια μία ένα ένας ενός υπό οποία οποίο οποίος οποίους δικό '
               'μετά κοντά έως εώς άλλους κάτι γύρω πιο όσο έχουν μπορώ μπορεί μπορείτε μπορούν πάνε κάντε δικό θέλετε '
               'δώσετε προς όπως δώστε δει δείτε βλέπω έτσι άλλοι ίδια νέα πολλά κυρίως άλλη ακόμα οποίων επί είπε όχι '
               'μέχρι μου σου του δυο δύο πλέον είπε α β γ δ ε ζ η θ ι κ λ μ ν ξ ο π ρ σ τ υ φ χ ψ ω αυτ όλη όσους '
               'όλες αυτούς θέλω βάζω κάθε κά τότε έχουμε θέλει έκανε βρίσκεται ακόμη όπου φέτος πέρσι μετά πέρυσι '
               'πήρε έδωσε ης βγει νέος νέα νέοι νέους νέας νέων ος γίνει υπάρχει υπάρχω υπάρχουν πάλι θέμα πώς μια μία μιας'
               'ανεξαρτήτως ίδιος μάλιστα έναν έγινε άλλο τρία τρια τρείς τρεις πέντε έξι επτά οκτώ δέκα τέσσερις θεμα'
               'ετ κεδ εναντίον τέσσερα αε ίδιες σ ς εννέα εννιά αο γς τέλος πάντως επίσης ας πχ x εκ περί αυτές άλλων'.split(' '))

s2 = set([word.upper() for word in stoplist])
s3 = set([word.capitalize() for word in stoplist])

stoplist = stoplist.union(s2)
stoplist = stoplist.union(s3)

In [ ]:
omilies = pd.read_csv('/content/drive/MyDrive/datasets/omilies.csv')
omilies.columns

In [ ]:
texts = omilies['pars'].tolist()

In [ ]:
sentence_model = SentenceTransformer("lighteternal/stsb-xlm-r-greek-transfer")  # lighteternal/stsb-xlm-r-greek-transfer , paraphrase-multilingual-MiniLM-L12-v2 , paraphrase-multilingual-mpnet-base-v2
embeddings = sentence_model.encode(texts, show_progress_bar=True)

In [ ]:
umap_model = UMAP(n_neighbors=25, n_components=5, min_dist=0.0, metric='cosine')
hdbscan_model = HDBSCAN(min_cluster_size=10, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

# vectorizer_model=vectorizer_model, umap_model=umap_model, hdbscan_model=hdbscan_model
vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words=list(stoplist), min_df=5, max_df=0.70)
topic_model = BERTopic(vectorizer_model=vectorizer_model, language="greek", calculate_probabilities=False, verbose=True, diversity=0.75, embedding_model=sentence_model)  # .fit_transform(texts)  
topics, probs = topic_model.fit_transform(texts, embeddings)

2022-11-09 05:12:35,912 - BERTopic - Reduced dimensionality
2022-11-09 05:12:47,577 - BERTopic - Clustered reduced embeddings


In [ ]:
topic_model.get_topic(2)

[('τουρκία', 0.012633784535577189),
 ('τουρκίας', 0.00995487609694794),
 ('ερντογάν', 0.007580281397730081),
 ('τουρκική', 0.005154587445246985),
 ('αιγαίο', 0.00419660331365808),
 ('συμπεριφορά', 0.004186285348572155),
 ('ανατολική μεσόγειο', 0.0039031347501354524),
 ('erdogan', 0.003667085413604224),
 ('μεσόγειο', 0.003543238066355413),
 ('ανατολική', 0.0035249062036160107)]

In [ ]:
renamed_topics = []
for i in range(1, 1697, 1):
  topic = ''
  for elem in topic_model.get_topic(i):
    topic += str(elem[0]) + " "
  renamed_topics.append(topic)

In [ ]:
topic_model.get_topic_freq()

# BERT NEWS CLASSIFICATION

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/datasets/datasetBERTCLASS_2.csv', encoding='utf8')
df.head()

,Unnamed: 0,text,category
0,0,άνοιξε σύστημα ηλεκτρονική αίτηση χορήγηση μετ...,Κοινωνία
1,1,τετάρτη ξεκίνησε διαδικασία μετεγγραφής φοιτητ...,Κοινωνία
2,2,κατασκευή στεγάστρου ανοιχτό κολυμβητήριο παλλ...,Κοινωνία
3,3,σοβαρή καταγγελία έρχεται φως δημοσιότητας αφο...,Κοινωνία
4,4,ψυχιατρική κλινική δρομοκαΐτειου νοσοκομείου ε...,Κοινωνία


In [ ]:
encoder = LabelEncoder()
df['categoryEncoded'] = encoder.fit_transform(df['category'])
df['tlen'] = df['text'].apply(lambda x: len(str(x).split()))

In [ ]:
dfp = df[df.text.notna()]

In [ ]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts,
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    return np.array(enc_di['input_ids'])

In [ ]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/bert-base-greek-uncased-v1")
model = AutoModel.from_pretrained("nlpaueb/bert-base-greek-uncased-v1")

In [ ]:
X_train,X_test ,y_train,y_test = train_test_split(df['text'], df['categoryEncoded'], random_state = 2020, test_size = 0.3)

In [ ]:
frame = {'text': X_train, 'categoryEncoded': y_train}
df = pd.DataFrame(frame)

import math
categories = [0,1,2,3,4,5,6,7,8,9,10,11]
sum = len(df[df['categoryEncoded']==8])
nn = df
for ctg in categories:
  dd = df[df['categoryEncoded']==ctg]
  subsum = len(dd)
  d = math.floor(sum / subsum)
  print(d)
  u = sum % subsum
  for i in range(d-1):
    nn = pd.concat([nn, dd])
  df2 = dd.head(u)
  nn = pd.concat([nn, df2])
len(nn)

X_train = nn['text'].squeeze()
y_train = nn['categoryEncoded'].squeeze()

In [ ]:
Xtrain_encoded = regular_encode(X_train.astype('str').to_list(), tokenizer, maxlen=256)
ytrain_encoded = tf.keras.utils.to_categorical(y_train, num_classes=12,dtype = 'int32')
Xtest_encoded = regular_encode(X_test.astype('str').to_list(), tokenizer, maxlen=256)
ytest_encoded = tf.keras.utils.to_categorical(y_test, num_classes=12,dtype = 'int32')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
def build_model(transformer, loss='categorical_crossentropy', max_len=512):
    input_word_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    #adding dropout layer
    x = tf.keras.layers.Dropout(0.3)(cls_token)
    #using a dense layer of 40 neurons as the number of unique categories is 12. 
    out = tf.keras.layers.Dense(12, activation='softmax')(x)
    model = tf.keras.Model(inputs=input_word_ids, outputs=out)
    #using categorical crossentropy as the loss as it is a multi-class classification problem
    model.compile(tf.keras.optimizers.Adam(lr=3e-5), loss=loss, metrics=['accuracy'])
    return model

In [ ]:
#building the model on tpu
with strategy.scope():
    transformer_layer = transformers.TFAutoModel.from_pretrained('nlpaueb/bert-base-greek-uncased-v1')
    model = build_model(transformer_layer, max_len=256)
model.summary()

In [ ]:
#creating the training and testing dataset.
BATCH_SIZE = 32*strategy.num_replicas_in_sync
AUTO = tf.data.experimental.AUTOTUNE 
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((Xtrain_encoded, ytrain_encoded))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)
test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(Xtest_encoded)
    .batch(BATCH_SIZE)
)

In [ ]:
#training for 10 epochs
n_steps = Xtrain_encoded.shape[0] // BATCH_SIZE
train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    epochs=5
)

In [ ]:
#making predictions
preds = model.predict(test_dataset,verbose = 1)
#converting the one hot vector output to a linear numpy array.
pred_classes = np.argmax(preds, axis = 1)

563/563 [==============================] - 44s 73ms/step


In [ ]:
#extracting the classes from the label encoder
encoded_classes = encoder.classes_
#mapping the encoded output to actual categories
predicted_category = [encoded_classes[x] for x in pred_classes]
true_category = [encoded_classes[x] for x in y_test]

In [ ]:
result_df = pd.DataFrame({'description':X_test,'true_category':true_category, 'predicted_category':predicted_category})
result_df.head()

,description,true_category,predicted_category
35948,κολλητή φίλη καρολάιν αποκαλύπτει «ο στάθης πα...,Κοινωνία,Κοινωνία
54937,πρώτη καταγγέλλουσα βήμα μάρτυρα κλειστές πόρτ...,Δικαιοσύνη,Κοινωνία
41269,υφυπουργός υποδομών μεταφορών αρμόδιος υποδομέ...,Πολιτική,Πολιτική
9849,αναρωτηθεί κάποιοι άνθρωποι προσελκύουν κουνού...,Υγεία,Περιβάλλον
17831,greek freak προανήγγειλε άφιξή tweet ελληνικά ...,Αθλητισμός,Τεχνολογία


In [ ]:
print(f"Accuracy is {sklearn.metrics.accuracy_score(result_df['true_category'], result_df['predicted_category'])}")

Accuracy is 0.8714841578654808


In [ ]:
result_df.to_csv('/content/drive/MyDrive/datasets/testPredictions.csv', index = False)

In [ ]:
result_df[result_df['true_category']!=result_df['predicted_category']]

,description,true_category,predicted_category
54937,πρώτη καταγγέλλουσα βήμα μάρτυρα κλειστές πόρτ...,Δικαιοσύνη,Κοινωνία
9849,αναρωτηθεί κάποιοι άνθρωποι προσελκύουν κουνού...,Υγεία,Περιβάλλον
18898,σύμφωνα έκτακτο δελτίο επιδείνωσης καιρού εκδό...,Κοινωνία,Περιβάλλον
1498,energean ανακοίνωσε ερευνητική γεώτρηση «ερμής...,Οικονομία,Κοινωνία
28718,δούμε επόμενο επεισόδιο σειράς «σασμός» προβλη...,Κοινωνία,Υγεία
...,...,...,...
27345,ακραίες θερμοκρασίες υψηλές χαμηλές τροφοδοτού...,Υγεία,Κόσμος
45113,«αποδημία» βασίλισσας ελισσάβετ καθοριστική επ...,Οικονομία,Κόσμος
688,παρασκευή κατατίθενται αιτήσεις περιοχές προτί...,Κοινωνία,Οικονομία
4406,υψηλός πληθωρισμός γερμανία αγγίζει ενεργειακή...,Κόσμος,Οικονομία


In [ ]:
confusion_mat = confusion_matrix(y_true = true_category, y_pred = predicted_category, labels=list(encoded_classes))

In [ ]:
df_cm = pd.DataFrame(confusion_mat, index = list(encoded_classes),columns = list(encoded_classes))
plt.rcParams['figure.figsize'] = (40,20)
sns.heatmap(df_cm, cmap='Blues', annot = True,fmt="d")

# COMBINATION TO CATEGORIZE TOPICS

In [ ]:
with open('/content/drive/MyDrive/datasets/omilies_topics.txt') as f:
    lines = f.readlines()

In [ ]:
lines[0] + lines[0]

'ανέφερε τόνισε σημείωσε πρόσθεσε τόνισε-πρωθυπουργός δήλωσε ανέφερε-πρωθυπουργός απ τόνισε-κυριάκος-μητσοτάκης τόνισε-κυριάκος \nανέφερε τόνισε σημείωσε πρόσθεσε τόνισε-πρωθυπουργός δήλωσε ανέφερε-πρωθυπουργός απ τόνισε-κυριάκος-μητσοτάκης τόνισε-κυριάκος \n'

In [ ]:
texts_list = []
for line in lines:
  line = line.replace('\n', '')
  line = line.replace('  ', '')
  texts_list.append(line)
frame = {'text': texts_list}
topics = pd.DataFrame(frame)

In [ ]:
topics.head()

,text
0,ανέφερε τόνισε σημείωσε πρόσθεσε τόνισε-πρωθυπ...
1,εμβόλια εμβόλιο εμβολίων εμβολιαστούν εμβολιασ...
2,προσφύγων ασύλου πρόσφυγες μεταναστών ροών ροέ...
3,τουρκία τουρκίας τουρκική ερντογάν συμπεριφορά...
4,υφυπουργός γραμματέας γενικός γενικός-γραμματέ...


In [ ]:
tops = pd.DataFrame(topics, columns=['text'])

In [ ]:
tops

,text
0,ανέφερε τόνισε σημείωσε πρόσθεσε τόνισε-πρωθυπ...
1,εμβόλια εμβόλιο εμβολίων εμβολιαστούν εμβολιασ...
2,προσφύγων ασύλου πρόσφυγες μεταναστών ροών ροέ...
3,τουρκία τουρκίας τουρκική ερντογάν συμπεριφορά...
4,υφυπουργός γραμματέας γενικός γενικός-γραμματέ...
...,...
1741,κοπεί εδώ-ευρώπη κίνα δεσμοί δημοσιογραφος οικ...
1742,ανοιχτά-θέματα σκληρών ψυχρού-πολέμου ψυχρού π...
1743,ελάτε απευθείας-μαζί νομίζω-δυνατότητα πείτε-π...
1744,συνολικό-ποσό μάη οφειλών αιτηθεί ρυθμίσει εκα...


In [ ]:
topics_encoded = regular_encode(tops['text'].astype('str').to_list(), tokenizer, maxlen=256)

In [ ]:
topics_dataset = (
    tf.data.Dataset
    .from_tensor_slices(topics_encoded)
    .batch(BATCH_SIZE)
)

In [ ]:
#making predictions
preds2 = model.predict(topics_dataset,verbose = 1)
#converting the one hot vector output to a linear numpy array.
pred_classes2 = np.argmax(preds2, axis = 1)

55/55 [==============================] - 4s 73ms/step


In [ ]:
predicted_category2 = [encoded_classes[x] for x in pred_classes2]

In [ ]:
result_topics = pd.DataFrame({'topic':topics.text.tolist(), 'predicted_category':predicted_category2})
result_topics.head()

,topic,predicted_category
0,ανέφερε τόνισε σημείωσε πρόσθεσε τόνισε-πρωθυπ...,Πολιτική
1,εμβόλια εμβόλιο εμβολίων εμβολιαστούν εμβολιασ...,Υγεία
2,προσφύγων ασύλου πρόσφυγες μεταναστών ροών ροέ...,Κοινωνία
3,τουρκία τουρκίας τουρκική ερντογάν συμπεριφορά...,Κόσμος
4,υφυπουργός γραμματέας γενικός γενικός-γραμματέ...,Πολιτική


In [ ]:
result_topics

,topic,predicted_category
0,ανέφερε τόνισε σημείωσε πρόσθεσε τόνισε-πρωθυπ...,Πολιτική
1,εμβόλια εμβόλιο εμβολίων εμβολιαστούν εμβολιασ...,Υγεία
2,προσφύγων ασύλου πρόσφυγες μεταναστών ροών ροέ...,Κοινωνία
3,τουρκία τουρκίας τουρκική ερντογάν συμπεριφορά...,Κόσμος
4,υφυπουργός γραμματέας γενικός γενικός-γραμματέ...,Πολιτική
...,...,...
1741,κοπεί εδώ-ευρώπη κίνα δεσμοί δημοσιογραφος οικ...,Πολιτική
1742,ανοιχτά-θέματα σκληρών ψυχρού-πολέμου ψυχρού π...,Κόσμος
1743,ελάτε απευθείας-μαζί νομίζω-δυνατότητα πείτε-π...,Πολιτική
1744,συνολικό-ποσό μάη οφειλών αιτηθεί ρυθμίσει εκα...,Οικονομία


In [ ]:
result_topics.predicted_category.unique()

array(['Πολιτική', 'Υγεία', 'Κοινωνία', 'Κόσμος', 'Οικονομία',
       'Τεχνολογία', 'Περιβάλλον', 'Πολιτισμός', 'Αθλητισμός',
       'Αστυνομικό', 'Εκπαίδευση', 'Δικαιοσύνη'], dtype=object)

In [ ]:
result_topics.to_csv('/content/drive/MyDrive/datasets/result_arthra_topics.csv')

In [ ]:
import torch
torch.save(model, '/content/drive/MyDrive/datasets/bertclf')

# saved_model = torch.load('path/to/model')

In [ ]:
for i in range(1, 1696, 1):
  topic_model.set_topic_labels({i: renamed_topics[i-1] + "_" + predicted_category2[i-1]})

In [ ]:
topic_model.custom_labels_

In [ ]:
keimena.pars.tolist()